# TimeGPT Quickstart

> Unlock the power of accurate predictions and confidently navigate uncertainty. Reduce uncertainty and resource limitations.
> With TimeGPT, you can effortlessly access state-of-the-art models to make data-driven decisions. Whether you're a bank
> forecasting market trends or a startup predicting product demand, TimeGPT democratizes access to cutting-edge predictive
> insights.

In [ ]:
#| hide
from nixtla.utils import colab_badge

In [ ]:
#| echo: false
colab_badge('docs/getting-started/2_quickstart')

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nixtla/nixtla/blob/main/nbs/docs/getting-started/2_quickstart.ipynb)

## Step 1: Create a TimeGPT account and generate your API key

* Go to [dashboard.nixtla.io](https://dashboard.nixtla.io/team/1245738/usage)
* Sign in with Google, GitHub or your email
* Create your API key by going to 'API Keys' in the menu and clicking on 'Create New API Key'
* Refresh the page and your API key will be there
    

## Step 2: Install Nixtla

You can install `nixtla` with `pip`:
    
```shell
pip install nixtla
```

## Step 3: Import the Nixtla TimeGPT client

In your favorite Python development environment:

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
load_dotenv()

In [ ]:
from nixtla import NixtlaClient

You can instantiate the `NixtlaClient` class providing your authentication API key. 

In [ ]:
nixtla_client = NixtlaClient(
    api_key = 'my_api_key_provided_by_nixtla'
)

In [ ]:
#| hide
nixtla_client = NixtlaClient()

Check your API key status with the `validate_api_key` method.

In [ ]:
nixtla_client.validate_api_key()

**This will get you started, but for more secure usage, see [Setting Up your API Key](https://docs.nixtla.io/docs/setting_up_your_api_key).**

## Step 4: Start making forecasts!

Now you can start making forecasts! Let's import an example on the classic `AirPassengers` dataset. This dataset contains the monthly number of airline passengers in Australia between 1949 and 1960. First, let's load the dataset and plot it:

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/air_passengers.csv')
df.head()

In [ ]:
nixtla_client.plot(df, time_col='timestamp', target_col='value')

:::{.callout-tip collapse=true}
## Important requirements of the data

* Make sure the target variable column does not have missing or non-numeric values.
* Do not include gaps/jumps in the datestamps (for the given frequency) between the first and late datestamps. The forecast function will not impute missing dates.
* The format of the datestamp column should be readable by Pandas (see [this link](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) for more details).
:::

Next, forecast the next 12 months using the SDK `forecast` method. Set the following parameters:

- `df`: A pandas dataframe containing the time series data.
- `h`: The number of steps ahead to forecast.
- `freq`: The frequency of the time series in Pandas format. See [pandas’ available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
- `time_col`: Column that identifies the datestamp column.
- `target_col`: The variable that we want to forecast.

In [ ]:
timegpt_fcst_df = nixtla_client.forecast(df=df, h=12, freq='MS', time_col='timestamp', target_col='value')
timegpt_fcst_df.head()

In [ ]:
nixtla_client.plot(df, timegpt_fcst_df, time_col='timestamp', target_col='value')

You can also produce longer forecasts increasing the horizon parameter and selecting the `timegpt-1-long-horizon` model. We recomend using this model if you want to predict more than one seasonal period of your data. 

For example, let's forecast the next 36 months:

In [ ]:
timegpt_fcst_df = nixtla_client.forecast(df=df, h=36, time_col='timestamp', target_col='value', freq='MS', model='timegpt-1-long-horizon')
timegpt_fcst_df.head()

In [ ]:
nixtla_client.plot(df, timegpt_fcst_df, time_col='timestamp', target_col='value')

You can also produce a shorter forecast. For this, we recommend using the default model, `timegpt-1`. 

In [ ]:
timegpt_fcst_df = nixtla_client.forecast(df=df, h=6, time_col='timestamp', target_col='value', freq='MS')
nixtla_client.plot(df, timegpt_fcst_df, time_col='timestamp', target_col='value')

::: {.callout-warning}
TimeGPT-1 is currently optimized for short horizon forecasting. While the `forecast` method will allow any positive and large horizons, the accuracy of the forecasts might degrade. We are currently working to improve the accuracy on longer forecasts.
::: 

## Using DateTime index to infer frequency

The freq parameter, which indicates the time unit between consecutive data points, is particularly critical. Fortunately, you can pass a DataFrame with a DateTime index to the forecasting method, ensuring that your time series data is equipped with necessary temporal features. By assigning a suitable freq parameter to the DateTime index of a DataFrame, you inform the model about the consistent interval between observations — be it days ('D'), months ('M'), or another suitable frequency.

In [ ]:
df_time_index = df.set_index('timestamp')
df_time_index.index = pd.DatetimeIndex(df_time_index.index, freq='MS')
nixtla_client.forecast(df=df, h=36, time_col='timestamp', target_col='value').head()